In [1]:
import math
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import os
import pickle
import matplotlib 
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import sklearn
from sklearn.model_selection import KFold

%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import imblearn as imb

from sklearn.metrics import confusion_matrix
import itertools

from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix, matthews_corrcoef,balanced_accuracy_score

from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

import sys
sys.path.append('../../')
# import utils
from sklearn.model_selection import GridSearchCV

from split_data import *
# from RF_Utils import *
# from RF_GSCV import *
from RF_atomver import *

In [2]:
def get_arrays(file_path, df_filename, filename_type=None, save=False):
    """use dataframes to get trainX, trainy, testX, testy out. Optional: save those files to csv
    file_path: directory
    df_filename: dataframe NEK#_binding_moe_{sampling}_df.csv (sampling: scaled, UNDER, SMOTE, ADASYN)
    split dataframe to train and test, and x and y
    save: bool, option to save splits to separate csv files (train X, train y, test X, test y) 
    returns: numpy arrays train X, train y, testX, test y"""
    df = pd.read_csv(file_path+df_filename)
    train_df= df[df['subset']=='train']
    test_df = df[df['subset']=='test']
    train_y = train_df['active'].to_numpy().reshape(-1)
    test_y=test_df['active'].to_numpy().reshape(-1)
    train_x_df = train_df.drop(columns='active')

  
    test_x_df = test_df.drop(columns='active')
    
    train_x_df = train_df.drop(columns='active')
    test_x_df = test_df.drop(columns='active')
    trainX = train_x_df.select_dtypes(include='number').to_numpy()
    testX = test_x_df.select_dtypes(include='number').to_numpy()
    
    print(f'train X shape: {trainX.shape}, y: {train_y.shape}, test X: {testX.shape}, y:{test_y.shape}')
    if (save and filename_type is not None): 
        trainxdf = pd.DataFrame(trainX)
        trainxdf.to_csv(file_path+filename_type+'_trainX.csv', index=False)
        # train_x_df.to_csv(filename_type+'_trainX.csv', index=False)
        trainy_df = pd.DataFrame(train_y)
        trainy_df.to_csv(file_path+filename_type+'_train_y.csv', index=False) 
        # test_x_df.to_csv(filename_type+'_testX.csv', index=False)
        testxdf = pd.DataFrame(testX)
        # testxdf.to_csv(file_path+filename_type+'_testX.csv', index=False)
        testy_df = pd.DataFrame(test_y)
        # testy_df.to_csv(file_path+filename_type+'_test_y.csv', index=False) 
        
    return trainX, train_y, testX, test_y

In [4]:
def add_cm(df): 
    # print(filepath)
    # print(filename)
    # df = pd.read_csv(filepath+filename)
    
    true_labels = df['y'] 
    predictions = df['prediction']
    cm = confusion_matrix(true_labels, predictions )
    cm_flattened = cm.flatten().tolist()
    df['cm'] = [cm_flattened]* len(df)
    df['prediction_type'] = df.apply(lambda x: prediction_type(x['y'], x['prediction']), axis=1)
    
    df['f1'] = f1_score(df['y'], df['prediction'])
    df['ROC-AUC'] = roc_auc_score(df['y'], df['prediction'])

    df['MCC'] = matthews_corrcoef(df['y'], df['prediction'])
    df['Balanced Accuracy'] = balanced_accuracy_score(df['y'], df['prediction'])
    return df
        

# Default RF 

In [5]:
data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'
nek_nums = [2,3,5,9]
# nek_nums = [2] # changed for NEK2 inhib
samplings = ['scaled', 'UNDER', 'SMOTE', 'ADASYN'] 
features = ['moe', 'mfp']
NEK= 'NEK'

for i, n in enumerate(nek_nums):
    nek = str(n)
    model_type = 'RF'
    nek_path= f'{data_dir}NEK{nek}/bind/'
 
    for k, feat in enumerate(features): 
        print()
        for j, samp in enumerate(samplings): 
            
            print(f'NEK{nek} bind {feat} {samp} {model_type}')
            file_root = f'NEK{nek}_binding_{feat}_{samp}_df.csv'
 
            trainX, trainy, testX, testy = get_arrays(nek_path,file_root)
            model = rf_models(trainX, trainy, testX, testy, model_type, {})
            train_df = gather_rf_results(model, trainX, trainy)
            test_df = gather_rf_results(model, testX, testy)
            train_df['subset'] = 'train' 
            test_df['subset'] = 'test' 
            model_name = f'NEK{nek}_binding_{feat}_{samp}_{model_type}' 
            train_df['model'] = model_name
            
            test_df['model'] = model_name
            train_df = add_cm(train_df)
            test_df = add_cm(test_df)
            
            train_df['NEK'] = f'NEK{nek}_binding'
            train_df['feat_type'] = feat
            train_df['strategy'] = samp
            train_df['RF_type'] = model_type
            test_df['NEK'] = f'NEK{nek}_binding'
            test_df['feat_type'] = feat
            test_df['strategy'] = samp
            test_df['RF_type'] = model_type
            
            with open(f'{model_name}.pkl', 'wb') as f: 
                pickle.dump(model,f)
            train_df.to_csv(f'{model_name}_train.csv',index=False) 
            test_df.to_csv(f'{model_name}_test.csv',index=False) 
            
            
            print()    
    # print()
    if n == 2 or n == 9:

        nek_path= f'{data_dir}NEK{nek}/inhib/'
        for k, feat in enumerate(features): 
            print()
            for j, samp in enumerate(samplings): 
                file_root = f'NEK{nek}_inhibition_{feat}_{samp}_df.csv'
                print(f'NEK{nek} inhib {feat} {samp} {model_type}')
                trainX, trainy, testX, testy = get_arrays(nek_path,file_root)
                model = rf_models(trainX, trainy, testX, testy, {model_type}, {})
                train_df = gather_rf_results(model, trainX, trainy)
                test_df = gather_rf_results(model, testX, testy)
                train_df['subset'] = 'train' 
                test_df['subset'] = 'test' 
                model_name = f'NEK{nek}_inhibition_{feat}_{samp}_{model_type}' 
                train_df['model'] = model_name
                test_df['model'] = model_name
                train_df = add_cm(train_df)
                test_df = add_cm(test_df)
                train_df['NEK'] = f'NEK{nek}_inhibition'
                train_df['feat_type'] = feat
                train_df['strategy'] = samp
                train_df['RF_type'] = model_type
                test_df['NEK'] = f'NEK{nek}_inhibition'
                test_df['feat_type'] = feat
                test_df['strategy'] = samp
                test_df['RF_type'] = model_type
                with open(f'{model_name}.pkl', 'wb') as f: 
                    pickle.dump(model, f) 
                train_df.to_csv(f'{model_name}_train.csv',index=False) 
                test_df.to_csv(f'{model_name}_test.csv',index=False) 
            
                print()
      
    print()


NEK2 bind moe scaled RF

NEK2 bind moe UNDER RF

NEK2 bind moe SMOTE RF

NEK2 bind moe ADASYN RF


NEK2 bind mfp scaled RF

NEK2 bind mfp UNDER RF

NEK2 bind mfp SMOTE RF

NEK2 bind mfp ADASYN RF


NEK2 inhib moe scaled RF
train X shape: (1624, 306), y: (1624,), test X: (408, 306), y:(408,)
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.949, precision: 0.652, recall: 0.536, specificity: 0.979

NEK2 inhib moe UNDER RF
train X shape: (224, 306), y: (224,), test X: (408, 306), y:(408,)
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.799, precision: 0.235, recall: 0.857, specificity: 0.795

NEK2 inhib moe SMOTE RF
train X shape: (3024, 306), y: (3024,), test X: (408, 306), y:(408,)
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.929, precision: 0.486, recall: 0.607, specificity: 0.953

NEK2 inhib moe ADASYN RF
train X shape: (3026, 306), y: (3026,), te

# RF + BCW 

In [6]:
data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'
nek_nums = [2,3,5,9] # changed for NEK2 inhib 
samplings = ['scaled', 'UNDER', 'SMOTE', 'ADASYN'] 
features = ['moe', 'mfp']
NEK= 'NEK'

for i, n in enumerate(nek_nums):
    nek = str(n)
    model_type = 'RF_BCW'
    nek_path= f'{data_dir}NEK{nek}/bind/'
 
    for k, feat in enumerate(features): 
        print()
        for j, samp in enumerate(samplings): 
            
            print(f'NEK{nek} bind {feat} {samp} {model_type}')
            file_root = f'NEK{nek}_binding_{feat}_{samp}_df.csv'
 
            trainX, trainy, testX, testy = get_arrays(nek_path,file_root)
            model = rf_models(trainX, trainy, testX, testy, model_type, {})
            train_df = gather_rf_results(model, trainX, trainy)
            test_df = gather_rf_results(model, testX, testy)
            train_df['subset'] = 'train' 
            test_df['subset'] = 'test' 
            model_name = f'NEK{nek}_binding_{feat}_{samp}_{model_type}' 
            train_df['model'] = model_name
            
            test_df['model'] = model_name
            train_df = add_cm(train_df)
            test_df = add_cm(test_df)
            
            train_df['NEK'] = f'NEK{nek}_binding'
            train_df['feat_type'] = feat
            train_df['strategy'] = samp
            train_df['RF_type'] = model_type
            test_df['NEK'] = f'NEK{nek}_binding'
            test_df['feat_type'] = feat
            test_df['strategy'] = samp
            test_df['RF_type'] = model_type
            
            with open(f'{model_name}.pkl', 'wb') as f: 
                pickle.dump(model,f)
            train_df.to_csv(f'{model_name}_train.csv',index=False) 
            test_df.to_csv(f'{model_name}_test.csv',index=False) 
            
            
            print()    
    # print()
    if n == 2 or n == 9:

        nek_path= f'{data_dir}NEK{nek}/inhib/'
        for k, feat in enumerate(features): 
            print()
            for j, samp in enumerate(samplings): 
                file_root = f'NEK{nek}_inhibition_{feat}_{samp}_df.csv'
                print(f'NEK{nek} inhib {feat} {samp} {model_type}')
                trainX, trainy, testX, testy = get_arrays(nek_path,file_root)
                model = rf_models(trainX, trainy, testX, testy, {model_type}, {})
                train_df = gather_rf_results(model, trainX, trainy)
                test_df = gather_rf_results(model, testX, testy)
                train_df['subset'] = 'train' 
                test_df['subset'] = 'test' 
                model_name = f'NEK{nek}_inhibition_{feat}_{samp}_{model_type}' 
                train_df['model'] = model_name
                test_df['model'] = model_name
                train_df = add_cm(train_df)
                test_df = add_cm(test_df)
                train_df['NEK'] = f'NEK{nek}_inhibition'
                train_df['feat_type'] = feat
                train_df['strategy'] = samp
                train_df['RF_type'] = model_type
                test_df['NEK'] = f'NEK{nek}_inhibition'
                test_df['feat_type'] = feat
                test_df['strategy'] = samp
                test_df['RF_type'] = model_type
                with open(f'{model_name}.pkl', 'wb') as f: 
                    pickle.dump(model, f) 
                train_df.to_csv(f'{model_name}_train.csv',index=False) 
                test_df.to_csv(f'{model_name}_test.csv',index=False) 
                

                print()
      
    print()


NEK2 bind moe scaled RF_BCW

NEK2 bind moe UNDER RF_BCW

NEK2 bind moe SMOTE RF_BCW

NEK2 bind moe ADASYN RF_BCW


NEK2 bind mfp scaled RF_BCW

NEK2 bind mfp UNDER RF_BCW

NEK2 bind mfp SMOTE RF_BCW

NEK2 bind mfp ADASYN RF_BCW


NEK2 inhib moe scaled RF_BCW
train X shape: (1624, 306), y: (1624,), test X: (408, 306), y:(408,)
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.946, precision: 0.636, recall: 0.500, specificity: 0.979

NEK2 inhib moe UNDER RF_BCW
train X shape: (224, 306), y: (224,), test X: (408, 306), y:(408,)
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.838, precision: 0.279, recall: 0.857, specificity: 0.837

NEK2 inhib moe SMOTE RF_BCW
train X shape: (3024, 306), y: (3024,), test X: (408, 306), y:(408,)
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.931, precision: 0.500, recall: 0.607, specificity: 0.955

NEK2 inhib moe ADASYN R

# BRFC

In [7]:
data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'
nek_nums = [2,3,5,9]
samplings = ['scaled', 'UNDER', 'SMOTE', 'ADASYN'] 
features = ['moe', 'mfp']
NEK= 'NEK'

for i, n in enumerate(nek_nums):
    nek = str(n)
    model_type = 'BRFC'
    nek_path= f'{data_dir}NEK{nek}/bind/'
 
    for k, feat in enumerate(features): 
        print()
        for j, samp in enumerate(samplings): 
            
            print(f'NEK{nek} bind {feat} {samp} {model_type}')
            file_root = f'NEK{nek}_binding_{feat}_{samp}_df.csv'
 
            trainX, trainy, testX, testy = get_arrays(nek_path,file_root)
            model = rf_models(trainX, trainy, testX, testy, model_type, {})
            train_df = gather_rf_results(model, trainX, trainy)
            test_df = gather_rf_results(model, testX, testy)
            train_df['subset'] = 'train' 
            test_df['subset'] = 'test' 
            model_name = f'NEK{nek}_binding_{feat}_{samp}_{model_type}' 
            train_df['model'] = model_name
            
            test_df['model'] = model_name
            train_df = add_cm(train_df)
            test_df = add_cm(test_df)
            
            train_df['NEK'] = f'NEK{nek}_binding'
            train_df['feat_type'] = feat
            train_df['strategy'] = samp
            train_df['RF_type'] = model_type
            test_df['NEK'] = f'NEK{nek}_binding'
            test_df['feat_type'] = feat
            test_df['strategy'] = samp
            test_df['RF_type'] = model_type
            
            with open(f'{model_name}.pkl', 'wb') as f: 
                pickle.dump(model,f)
            train_df.to_csv(f'{model_name}_train.csv',index=False) 
            test_df.to_csv(f'{model_name}_test.csv',index=False) 
            
            
            print()    
    # print()
    if n == 2 or n == 9:

        nek_path= f'{data_dir}NEK{nek}/inhib/'
        for k, feat in enumerate(features): 
            print()
            for j, samp in enumerate(samplings): 
                file_root = f'NEK{nek}_inhibition_{feat}_{samp}_df.csv'
                print(f'NEK{nek} inhib {feat} {samp} {model_type}')
                trainX, trainy, testX, testy = get_arrays(nek_path,file_root)
                model = rf_models(trainX, trainy, testX, testy, {model_type}, {})
                train_df = gather_rf_results(model, trainX, trainy)
                test_df = gather_rf_results(model, testX, testy)
                train_df['subset'] = 'train' 
                test_df['subset'] = 'test' 
                model_name = f'NEK{nek}_inhibition_{feat}_{samp}_{model_type}' 
                train_df['model'] = model_name
                test_df['model'] = model_name
                train_df = add_cm(train_df)
                test_df = add_cm(test_df)
                train_df['NEK'] = f'NEK{nek}_inhibition'
                train_df['feat_type'] = feat
                train_df['strategy'] = samp
                train_df['RF_type'] = model_type
                test_df['NEK'] = f'NEK{nek}_inhibition'
                test_df['feat_type'] = feat
                test_df['strategy'] = samp
                test_df['RF_type'] = model_type
                with open(f'{model_name}.pkl', 'wb') as f: 
                    pickle.dump(model, f) 
                train_df.to_csv(f'{model_name}_train.csv',index=False) 
                test_df.to_csv(f'{model_name}_test.csv',index=False) 
                

                print()
      
    print()


NEK2 bind moe scaled BRFC

NEK2 bind moe UNDER BRFC

NEK2 bind moe SMOTE BRFC

NEK2 bind moe ADASYN BRFC


NEK2 bind mfp scaled BRFC

NEK2 bind mfp UNDER BRFC

NEK2 bind mfp SMOTE BRFC

NEK2 bind mfp ADASYN BRFC


NEK2 inhib moe scaled BRFC
train X shape: (1624, 306), y: (1624,), test X: (408, 306), y:(408,)
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.944, precision: 0.609, recall: 0.500, specificity: 0.976

NEK2 inhib moe UNDER BRFC
train X shape: (224, 306), y: (224,), test X: (408, 306), y:(408,)
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.819, precision: 0.255, recall: 0.857, specificity: 0.816

NEK2 inhib moe SMOTE BRFC
train X shape: (3024, 306), y: (3024,), test X: (408, 306), y:(408,)
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.926, precision: 0.472, recall: 0.607, specificity: 0.950

NEK2 inhib moe ADASYN BRFC
train X shape: (30

# BRFC+BCW

In [8]:
data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'
# nek_nums = [2,3,5,9]
samplings = ['scaled', 'UNDER', 'SMOTE', 'ADASYN'] 
features = ['moe', 'mfp']
NEK= 'NEK'

for i, n in enumerate(nek_nums):
    nek = str(n)
    model_type = 'BRFC_BCW'
    nek_path= f'{data_dir}NEK{nek}/bind/'
 
    for k, feat in enumerate(features): 
        print()
        for j, samp in enumerate(samplings): 
            
            print(f'NEK{nek} bind {feat} {samp} {model_type}')
            # file_root = f'NEK{nek}_binding_{feat}_{samp}_df.csv'
            trainX, trainy, testX, testy = get_arrays(nek_path,file_root)
            model = rf_models(trainX, trainy, testX, testy, model_type, {})
            train_df = gather_rf_results(model, trainX, trainy)
            test_df = gather_rf_results(model, testX, testy)
            train_df['subset'] = 'train' 
            test_df['subset'] = 'test' 
            model_name = f'NEK{nek}_binding_{feat}_{samp}_{model_type}' 
            train_df['model'] = model_name
            
            test_df['model'] = model_name
            train_df = add_cm(train_df)
            test_df = add_cm(test_df)
            
            train_df['NEK'] = f'NEK{nek}_binding'
            train_df['feat_type'] = feat
            train_df['strategy'] = samp
            train_df['RF_type'] = model_type
            test_df['NEK'] = f'NEK{nek}_binding'
            test_df['feat_type'] = feat
            test_df['strategy'] = samp
            test_df['RF_type'] = model_type
            
            with open(f'{model_name}.pkl', 'wb') as f: 
                pickle.dump(model,f)
            train_df.to_csv(f'{model_name}_train.csv',index=False) 
            test_df.to_csv(f'{model_name}_test.csv',index=False) 
            
            
            print()    
    # print()
    if n == 2 or n == 9:

        nek_path= f'{data_dir}NEK{nek}/inhib/'
        for k, feat in enumerate(features): 
            print()
            for j, samp in enumerate(samplings): 
                file_root = f'NEK{nek}_inhibition_{feat}_{samp}_df.csv'
                print(f'NEK{nek} inhib {feat} {samp} {model_type}')
                trainX, trainy, testX, testy = get_arrays(nek_path,file_root)
                model = rf_models(trainX, trainy, testX, testy, {model_type}, {})
                train_df = gather_rf_results(model, trainX, trainy)
                test_df = gather_rf_results(model, testX, testy)
                train_df['subset'] = 'train' 
                test_df['subset'] = 'test' 
                model_name = f'NEK{nek}_inhibition_{feat}_{samp}_{model_type}' 
                train_df['model'] = model_name
                test_df['model'] = model_name
                train_df = add_cm(train_df)
                test_df = add_cm(test_df)
                train_df['NEK'] = f'NEK{nek}_inhibition'
                train_df['feat_type'] = feat
                train_df['strategy'] = samp
                train_df['RF_type'] = model_type
                test_df['NEK'] = f'NEK{nek}_inhibition'
                test_df['feat_type'] = feat
                test_df['strategy'] = samp
                test_df['RF_type'] = model_type
                with open(f'{model_name}.pkl', 'wb') as f: 
                    pickle.dump(model, f) 
                train_df.to_csv(f'{model_name}_train.csv',index=False) 
                test_df.to_csv(f'{model_name}_test.csv',index=False) 
                print()
      
    print()


NEK2 bind moe scaled BRFC_BCW

NEK2 bind moe UNDER BRFC_BCW

NEK2 bind moe SMOTE BRFC_BCW

NEK2 bind moe ADASYN BRFC_BCW


NEK2 bind mfp scaled BRFC_BCW

NEK2 bind mfp UNDER BRFC_BCW

NEK2 bind mfp SMOTE BRFC_BCW

NEK2 bind mfp ADASYN BRFC_BCW


NEK2 inhib moe scaled BRFC_BCW
train X shape: (1624, 306), y: (1624,), test X: (408, 306), y:(408,)
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.944, precision: 0.609, recall: 0.500, specificity: 0.976

NEK2 inhib moe UNDER BRFC_BCW
train X shape: (224, 306), y: (224,), test X: (408, 306), y:(408,)
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.826, precision: 0.258, recall: 0.821, specificity: 0.826

NEK2 inhib moe SMOTE BRFC_BCW
train X shape: (3024, 306), y: (3024,), test X: (408, 306), y:(408,)
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.929, precision: 0.486, recall: 0.607, specificity: 0.953

N

In [11]:

source = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/'
# dest = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_pickles/'
dest = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/'

if not os.path.exists(source):
    os.makedirs(dest)

for root, dirs, files in os.walk(source):
    for file in files:
        if file.endswith('.csv'):
            source_file = os.path.join(root, file)
            dest_file = os.path.join(dest, file)
            shutil.move(source_file,dest_file)
            print(f"Moved: {source_file} to {dest_file}")
        # if file.endswith('RF_BCW.pkl'):
        #     source_file = os.path.join(root, file)
        #     dest_file = os.path.join(dest, file)
        #     shutil.move(source_file,dest_file)
        #     print(f"Moved: {source_file} to {dest_file}")
        # if file.endswith('BRFC.pkl'):
        #     source_file = os.path.join(root, file)
        #     dest_file = os.path.join(dest, file)
        #     shutil.move(source_file,dest_file)
        #     print(f"Moved: {source_file} to {dest_file}")
        # if file.endswith('BRFC_BCW.pkl'):
        #     source_file = os.path.join(root, file)
        #     dest_file = os.path.join(dest, file)
        #     shutil.move(source_file,dest_file)
        #     print(f"Moved: {source_file} to {dest_file}")



Moved: /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/NEK2_binding_mfp_ADASYN_RF_GS_train_perf.csv to /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/NEK2_binding_mfp_ADASYN_RF_GS_train_perf.csv
Moved: /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/NEK2_inhibition_moe_ADASYN_BRFC_test.csv to /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/NEK2_inhibition_moe_ADASYN_BRFC_test.csv
Moved: /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/NEK2_binding_mfp_UNDER_RF_GS_train_perf.csv to /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/NEK2_binding_mfp_UNDER_RF_GS_train_perf.csv
Moved: /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/NEK2_binding_mfp_SMOTE_RF_GS_train_perf.csv to /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/NEK2_binding_mfp_SMOTE_RF_GS_tra

In [10]:

source = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/'
dest = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/'

if not os.path.exists(source):
    os.makedirs(dest)

for root, dirs, files in os.walk(source):
    for file in files:
        if file.endswith('.csv'):
            source_file = os.path.join(root, file)
            dest_file = os.path.join(dest, file)
            shutil.move(source_file,dest_file)
            print(f"Moved: {source_file} to {dest_file}")
        # if file.endswith('RF_BCW.csv'):
        #     source_file = os.path.join(root, file)
        #     dest_file = os.path.join(dest, file)
        #     shutil.move(source_file,dest_file)
        #     print(f"Moved: {source_file} to {dest_file}")
        # if file.endswith('BRFC.csv'):
        #     source_file = os.path.join(root, file)
        #     dest_file = os.path.join(dest, file)
        #     shutil.move(source_file,dest_file)
        #     print(f"Moved: {source_file} to {dest_file}")
        # if file.endswith('BRFC_BCW.csv'):
        #     source_file = os.path.join(root, file)
        #     dest_file = os.path.join(dest, file)
        #     shutil.move(source_file,dest_file)
        #     print(f"Moved: {source_file} to {dest_file}")



Moved: /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/NEK2_inhibition_moe_ADASYN_BRFC_test.csv to /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/NEK2_inhibition_moe_ADASYN_BRFC_test.csv
Moved: /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/NEK2_inhibition_moe_scaled_RF_train.csv to /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/NEK2_inhibition_moe_scaled_RF_train.csv
Moved: /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/NEK2_inhibition_mfp_UNDER_BRFC_BCW_train.csv to /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/NEK2_inhibition_mfp_UNDER_BRFC_BCW_train.csv
Moved: /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/NEK2_inhibition_mfp_scaled_RF_test.csv to /Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/NEK2_inhibition_mfp_scaled_RF_test.csv
Moved: /Users/jayceepang/msse/capstone/atom2024/at

In [48]:
neks = 6 

samplings = 4 
feats = 2 
rf_types = 4 
6*16*4


384

In [37]:
# data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/'
# nek_nums = [2,3,5,9]
# samplings = ['scaled', 'UNDER', 'SMOTE', 'ADASYN'] 
# features = ['moe', 'mfp']
# NEK= 'NEK'
# rf_types = ['RF', 'RF_BCW', 'BRFC', 'BRFC_BCW']
# for i, n in enumerate(nek_nums):
#     nek = str(n)

#     nek_path= f'{data_dir}NEK{nek}/bind/'
 
#     for k, feat in enumerate(features): 
#         print()
#         for j, samp in enumerate(samplings): 
#             for rf in rf_types: 
            
#                 print(f'NEK{nek} bind {feat} {samp} {model_type}')
#                 file_root = f'NEK{nek}_binding_{feat}_{samp}_{rf}'
#                 train_df = pd.read_csv(data_dir+file_root+'_train.csv')
#                 test_df = pd.read_csv(data_dir+file_root+'_test.csv')
#                 display(test_df.head(3))
#                 train_results = 
             

                
#                 print()    
#         # print()
#         if n == 2 or n == 9:
    
#             nek_path= f'{data_dir}NEK{nek}/inhib/'
#             for k, feat in enumerate(features): 
#                 print()
#                 for j, samp in enumerate(samplings): 
#                     for rf in rf_types: 
#                         print(f'NEK{nek} bind {feat} {samp} {model_type}')
#                         file_root = f'NEK{nek}_binding_{feat}_{samp}_{rf}'
#                         train_df = pd.read_csv(data_dir+file_root+'_train.csv')
#                         test_df = pd.read_csv(data_dir+file_root+'_test.csv')
             
#                     print()
          
#         print()


NEK2 bind moe scaled BRFC_BCW


,prediction,accuracy,precision,recall,specificity,prob_class0,prob_class1,y,TN,FN,FP,TP,subset,model,cm,prediction_type,NEK,feat_type,strategy,RF_type
0,0,0.957597,0.0,0.0,1.0,1.00,0.00,0,271,12,0,0,test,NEK2_binding_moe_scaled_RF,"[271, 0, 12, 0]",TN,NEK2,moe,scaled,RF
1,0,0.957597,0.0,0.0,1.0,0.97,0.03,0,271,12,0,0,test,NEK2_binding_moe_scaled_RF,"[271, 0, 12, 0]",TN,NEK2,moe,scaled,RF
2,0,0.957597,0.0,0.0,1.0,1.00,0.00,0,271,12,0,0,test,NEK2_binding_moe_scaled_RF,"[271, 0, 12, 0]",TN,NEK2,moe,scaled,RF



NEK2 bind moe scaled BRFC_BCW


,prediction,accuracy,precision,recall,specificity,prob_class0,prob_class1,y,TN,FN,FP,TP,subset,model,cm,prediction_type,NEK,feat_type,strategy,RF_type
0,0,0.957597,0.0,0.0,1.0,1.0,0.0,0,271,12,0,0,test,NEK2_binding_moe_scaled_RF_BCW,"[271, 0, 12, 0]",TN,NEK2,moe,scaled,RF_BCW
1,0,0.957597,0.0,0.0,1.0,1.0,0.0,0,271,12,0,0,test,NEK2_binding_moe_scaled_RF_BCW,"[271, 0, 12, 0]",TN,NEK2,moe,scaled,RF_BCW
2,0,0.957597,0.0,0.0,1.0,1.0,0.0,0,271,12,0,0,test,NEK2_binding_moe_scaled_RF_BCW,"[271, 0, 12, 0]",TN,NEK2,moe,scaled,RF_BCW



NEK2 bind moe scaled BRFC_BCW


...


NEK9 bind mfp ADASYN BRFC_BCW


,prediction,accuracy,precision,recall,specificity,prob_class0,prob_class1,y,TN,FN,FP,TP,subset,model,cm,prediction_type,NEK,feat_type,strategy,RF_type
0,0,0.954064,0.5,0.307692,0.985185,1.00,0.00,0,266,9,4,4,test,NEK9_binding_mfp_ADASYN_BRFC,"[266, 4, 9, 4]",TN,NEK9,mfp,ADASYN,BRFC
1,0,0.954064,0.5,0.307692,0.985185,1.00,0.00,0,266,9,4,4,test,NEK9_binding_mfp_ADASYN_BRFC,"[266, 4, 9, 4]",TN,NEK9,mfp,ADASYN,BRFC
2,0,0.954064,0.5,0.307692,0.985185,0.99,0.01,0,266,9,4,4,test,NEK9_binding_mfp_ADASYN_BRFC,"[266, 4, 9, 4]",TN,NEK9,mfp,ADASYN,BRFC



NEK9 bind mfp ADASYN BRFC_BCW


,prediction,accuracy,precision,recall,specificity,prob_class0,prob_class1,y,TN,FN,FP,TP,subset,model,cm,prediction_type,NEK,feat_type,strategy,RF_type
0,0,0.957597,0.571429,0.307692,0.988889,1.00,0.00,0,267,9,3,4,test,NEK9_binding_mfp_ADASYN_BRFC_BCW,"[267, 3, 9, 4]",TN,NEK9,mfp,ADASYN,BRFC_BCW
1,0,0.957597,0.571429,0.307692,0.988889,1.00,0.00,0,267,9,3,4,test,NEK9_binding_mfp_ADASYN_BRFC_BCW,"[267, 3, 9, 4]",TN,NEK9,mfp,ADASYN,BRFC_BCW
2,0,0.957597,0.571429,0.307692,0.988889,0.95,0.05,0,267,9,3,4,test,NEK9_binding_mfp_ADASYN_BRFC_BCW,"[267, 3, 9, 4]",TN,NEK9,mfp,ADASYN,BRFC_BCW




NEK9 bind moe scaled BRFC_BCW
NEK9 bind moe scaled BRFC_BCW
NEK9 bind moe scaled BRFC_BCW
NEK9 bind moe scaled BRFC_BCW

NEK9 bind moe UNDER BRFC_BCW
NEK9 bind moe UNDER BRFC_BCW
NEK9 bind moe UNDER BRFC_BCW
NEK9 bind moe UNDER BRFC_BCW

NEK9 bind moe SMOTE BRFC_BCW
NEK9 bind moe SMOTE BRFC_BCW
NEK9 bind moe SMOTE BRFC_BCW
NEK9 bind moe SMOTE BRFC_BCW

NEK9 bind moe ADASYN BRFC_BCW
NEK9 bind moe ADASYN BRFC_BCW
NEK9 bind moe ADASYN BRFC_BCW
NEK9 bind moe ADASYN BRFC_BCW


NEK9 bind mfp scaled BRFC_BCW
NEK9 bind mfp scaled BRFC_BCW
NEK9 bind mfp scaled BRFC_BCW
NEK9 bind mfp scaled BRFC_BCW

NEK9 bind mfp UNDER BRFC_BCW
NEK9 bind mfp UNDER BRFC_BCW
NEK9 bind mfp UNDER BRFC_BCW
NEK9 bind mfp UNDER BRFC_BCW

NEK9 bind mfp SMOTE BRFC_BCW
NEK9 bind mfp SMOTE BRFC_BCW
NEK9 bind mfp SMOTE BRFC_BCW
NEK9 bind mfp SMOTE BRFC_BCW

NEK9 bind mfp ADASYN BRFC_BCW
NEK9 bind mfp ADASYN BRFC_BCW
NEK9 bind mfp ADASYN BRFC_BCW
NEK9 bind mfp ADASYN BRFC_BCW




In [45]:
monthly = 50+50+40 
monthly*3

420

In [8]:
def other_metrics(df): 
    
    df['ROC-AUC'] = roc_auc_score(df['y'], df['prediction'])
    df['f1'] = f1_score(df['y'], df['prediction'])
    df['MCC'] = matthews_corrcoef(df['y'], df['prediction'])
    df['Balanced Accuracy'] = balanced_accuracy_score(df['y'], df['prediction'])
    return df 

In [15]:
neks = ['NEK2_binding', 'NEK2_inhibition', 'NEK3_binding', 'NEK5_binding', 'NEK9_binding', 'NEK9_inhibition'] 
nek_list = ["2", "3", "5", "9"]
nektype = ['binding','inhibition']
feat_types = ['moe', 'mfp']
samplings = ['scaled', 'UNDER' , 'SMOTE', 'ADASYN']
model_types = ['RF','RF_BCW', 'BRFC', 'BRFC_BCW']
count = 0 
pickle_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_pickles/'
result_path =  '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/'

for n in nek_list:
    for i in nektype: 
        for feat in feat_types: 
            for samp in samplings:  
                for rf in model_types:  # RF, RF_BCW, BRFC, BRFC_BCW
                    if i == 'binding':
                        data_path = f'/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK{n}/bind/'
                    
                        file_root = f'NEK{n}_{i}_{feat}_{samp}_{rf}'
                        count+=1
                    
                        train = pd.read_csv(f'{result_path}{file_root}_train.csv') 
                        
                        test = pd.read_csv(f'{result_path}{file_root}_test.csv') 
                        train = other_metrics(train) 
                        test=other_metrics(test) 
                        train.to_csv(f'{result_path}{file_root}_train.csv', index=False) 
                        test.to_csv(f'{result_path}{file_root}_test.csv', index=False) 
                        model_name = f'NEK{n}_{i}_{feat}_{samp}_{rf}'
                        print(f'{count}. {model_name}')
               
                    if n in ['2', '9'] and i == 'inhibition': 
                        file_root = f'NEK{n}_{i}_{feat}_{samp}_{rf}'
                        count+=1
                    
                        train = pd.read_csv(f'{result_path}{file_root}_train.csv') 
                        test = pd.read_csv(f'{result_path}{file_root}_test.csv') 
                        model_name = f'NEK{n}_{i}_{feat}_{samp}_{rf}'
                        train = other_metrics(train) 
                        test=other_metrics(test) 
                        train.to_csv(f'{result_path}{file_root}_train.csv', index=False) 
                        test.to_csv(f'{result_path}{file_root}_test.csv', index=False) 
                        print(f'{count}. {model_name}')
                
                





1. NEK2_binding_moe_scaled_RF
2. NEK2_binding_moe_scaled_RF_BCW
3. NEK2_binding_moe_scaled_BRFC
4. NEK2_binding_moe_scaled_BRFC_BCW
5. NEK2_binding_moe_UNDER_RF
6. NEK2_binding_moe_UNDER_RF_BCW
7. NEK2_binding_moe_UNDER_BRFC
8. NEK2_binding_moe_UNDER_BRFC_BCW
9. NEK2_binding_moe_SMOTE_RF
10. NEK2_binding_moe_SMOTE_RF_BCW
11. NEK2_binding_moe_SMOTE_BRFC
12. NEK2_binding_moe_SMOTE_BRFC_BCW
13. NEK2_binding_moe_ADASYN_RF
14. NEK2_binding_moe_ADASYN_RF_BCW
15. NEK2_binding_moe_ADASYN_BRFC
16. NEK2_binding_moe_ADASYN_BRFC_BCW
17. NEK2_binding_mfp_scaled_RF
18. NEK2_binding_mfp_scaled_RF_BCW
19. NEK2_binding_mfp_scaled_BRFC
20. NEK2_binding_mfp_scaled_BRFC_BCW
21. NEK2_binding_mfp_UNDER_RF
22. NEK2_binding_mfp_UNDER_RF_BCW
23. NEK2_binding_mfp_UNDER_BRFC
24. NEK2_binding_mfp_UNDER_BRFC_BCW
25. NEK2_binding_mfp_SMOTE_RF
26. NEK2_binding_mfp_SMOTE_RF_BCW
27. NEK2_binding_mfp_SMOTE_BRFC
28. NEK2_binding_mfp_SMOTE_BRFC_BCW
29. NEK2_binding_mfp_ADASYN_RF
30. NEK2_binding_mfp_ADASYN_RF_BCW
31. NEK